In [390]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import naive_bayes,model_selection,metrics
import xgboost as xgb
import string
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.emsemble import RandomForestClassifier, GradientBoostingClassifier

ModuleNotFoundError: No module named 'sklearn.emsemble'

In [282]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head(50)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0,0,1,0,0,0
6,7,On the rotation period and shape of the hyperb...,We observed the newly discovered hyperbolic ...,0,1,0,0,0,0
7,8,Adverse effects of polymer coating on heat tra...,The ability of metallic nanoparticles to sup...,0,1,0,0,0,0
8,9,SPH calculations of Mars-scale collisions: the...,We model large-scale ($\approx$2000km) impac...,0,1,0,0,0,0
9,10,$\mathcal{R}_{0}$ fails to predict the outbrea...,Time varying susceptibility of host at indiv...,0,0,0,0,1,0


In [280]:
train.shape

(20972, 9)

In [281]:
# i = 1
# train['target'] = 0
# for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
#      train[col] = train[col]*i
#      i+=1
# for j in range(0,train.shape[0]):
#     for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
#         if train.loc[j,[col]].values!=0:
#             train.loc[j,['target']] = train.loc[j,[col]].values
# # f = lambda x: 
    


In [130]:
train.head(10)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0,0,1,0,0,0
6,7,On the rotation period and shape of the hyperb...,We observed the newly discovered hyperbolic ...,0,1,0,0,0,0
7,8,Adverse effects of polymer coating on heat tra...,The ability of metallic nanoparticles to sup...,0,1,0,0,0,0
8,9,SPH calculations of Mars-scale collisions: the...,We model large-scale ($\approx$2000km) impac...,0,1,0,0,0,0
9,10,$\mathcal{R}_{0}$ fails to predict the outbrea...,Time varying susceptibility of host at indiv...,0,0,0,0,1,0


# Meta Features

In [283]:
train['num_words_title'] = train['TITLE'].apply(lambda x: len(str(x).split()))
test['num_words_title'] = test['TITLE'].apply(lambda x: len(str(x).split()))
train['num_words_abs'] = train['ABSTRACT'].apply(lambda x: len(str(x).split()))
test['num_words_abs'] = test['ABSTRACT'].apply(lambda x: len(str(x).split()))

In [284]:
train['num_unique_title'] = train['TITLE'].apply(lambda x: len(set(str(x).split())))
test['num_unique_title'] = test['TITLE'].apply(lambda x: len(set(str(x).split())))
train['num_unique_abs'] = train['ABSTRACT'].apply(lambda x: len(set(str(x).split())))
test['num_unique_abs'] = test['ABSTRACT'].apply(lambda x: len(set(str(x).split())))

In [285]:
train['word_upper_title'] = train['TITLE'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test['word_upper_title'] = test['TITLE'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
train['word_upper_abs'] = train['ABSTRACT'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test['word_upper_abs'] = test['ABSTRACT'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [286]:
train['word_title_title'] = train['TITLE'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test['word_title_title'] = test['TITLE'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
train['word_title_abs'] = train['ABSTRACT'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test['word_title_abs'] = test['ABSTRACT'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

In [287]:
train["num_punctuations"] =train['ABSTRACT'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['ABSTRACT'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

In [288]:
train["mean_word_len_tit"] = train["TITLE"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len_tit"] = test['TITLE'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train["mean_word_len_abs"] = train["ABSTRACT"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len_abs"] = test['ABSTRACT'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [289]:
train["num_chars"] = train["ABSTRACT"].apply(lambda x: len(str(x)))
test["num_chars"] = test["ABSTRACT"].apply(lambda x: len(str(x)))

In [290]:
tfidf_vec_t = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf_t = tfidf_vec_t.fit_transform(train['TITLE'].values.tolist() + test['TITLE'].values.tolist())
train_tfidf_t = tfidf_vec_t.transform(train['TITLE'].values.tolist())
test_tfidf_t = tfidf_vec_t.transform(test['TITLE'].values.tolist())
tfidf_vec_a = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf_a = tfidf_vec_a.fit_transform(train['ABSTRACT'].values.tolist() + test['ABSTRACT'].values.tolist())
train_tfidf_a = tfidf_vec_a.transform(train['ABSTRACT'].values.tolist())
test_tfidf_a = tfidf_vec_a.transform(test['ABSTRACT'].values.tolist())

In [292]:
n_t = 10
n_a =25
svd_t = TruncatedSVD(n_components=n_t,algorithm='arpack')
svd_a = TruncatedSVD(n_components=n_a,algorithm='arpack')
svd_t.fit(full_tfidf_t)
svd_a.fit(full_tfidf_a)
train_svd_t = pd.DataFrame(svd_t.transform(train_tfidf_t))
test_svd_t = pd.DataFrame(svd_t.transform(test_tfidf_t))
train_svd_a = pd.DataFrame(svd_a.transform(train_tfidf_a))
test_svd_a = pd.DataFrame(svd_a.transform(test_tfidf_a))
train_svd_t.columns = ['svd_word_t'+str(i) for i in range(n_t)]
test_svd_t.columns = ['svd_word_t'+str(i) for i in range(n_t)]
train = pd.concat([train, train_svd_t], axis=1)
test = pd.concat([test, test_svd_t], axis=1)
train_svd_a.columns = ['svd_word_a'+str(i) for i in range(n_a)]
test_svd_a.columns = ['svd_word_a'+str(i) for i in range(n_a)]
train = pd.concat([train, train_svd_a], axis=1)
test = pd.concat([test, test_svd_a], axis=1)
# del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [293]:
def mnb(train_x,train_y,test_x,test_y,test_x2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_x,train_y)
    pred_test_y = model.predict_proba(test_x)
    pred_test_y2 = model.predict_proba(test_x2)
    return pred_test_y, pred_test_y2, model
    

In [294]:
tfidf_vec_t = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec_t.fit(train['TITLE'].values.tolist() + test['TITLE'].values.tolist())
train_tfidf_t = tfidf_vec_t.transform(train['TITLE'].values.tolist())
test_tfidf_t = tfidf_vec_t.transform(test['TITLE'].values.tolist())
tfidf_vec_a = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec_a.fit(train['ABSTRACT'].values.tolist() + test['ABSTRACT'].values.tolist())
train_tfidf_a = tfidf_vec_a.transform(train['ABSTRACT'].values.tolist())
test_tfidf_a = tfidf_vec_a.transform(test['ABSTRACT'].values.tolist())

In [295]:
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    pred_full_test_t = 0
    pred_train_t = np.zeros([train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_tfidf_t[dev_index], train_tfidf_t[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = mnb(dev_X, dev_y, val_X, val_y, test_tfidf_t)
        pred_full_test_t = pred_full_test_t + pred_test_y
        pred_train_t[val_index,:] = pred_val_y
#         cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
    pred_full_test_t = pred_full_test_t / 5
    
    pred_full_test_a = 0
    pred_train_a = np.zeros([train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_tfidf_a[dev_index], train_tfidf_a[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = mnb(dev_X, dev_y, val_X, val_y, test_tfidf_a)
        pred_full_test_a = pred_full_test_a + pred_test_y
        pred_train_a[val_index,:] = pred_val_y
#         cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
    pred_full_test_a = pred_full_test_a / 5
    train["nb_t_"+col] = pred_train_t[:,0]
    train["nb_t_"+col] = pred_train_t[:,1]
#     train_df["nb_cvec_mws"] = pred_train[:,2]
    test["nb_t_"+col] = pred_full_test_t[:,0]
    test["nb_t_"+col] = pred_full_test_t[:,1]

    train["nb_a_"+col] = pred_train_a[:,0]
    train["nb_a_"+col] = pred_train_a[:,1]
#     train_df["nb_cvec_mws"] = pred_train[:,2]
    test["nb_a_"+col] = pred_full_test_a[:,0]
    test["nb_a_"+col] = pred_full_test_a[:,1]
    
    
#     test_df["nb_cvec_mws"] = pred_full_test[:,2]

In [296]:
tfidf_vec_t_2 = CountVectorizer(ngram_range=(1,3),analyzer='char')
tfidf_vec_t_2.fit(train['TITLE'].values.tolist() + test['TITLE'].values.tolist())
train_tfidf_t_2 = tfidf_vec_t_2.transform(train['TITLE'].values.tolist())
test_tfidf_t_2 = tfidf_vec_t_2.transform(test['TITLE'].values.tolist())
tfidf_vec_a_2 = CountVectorizer(ngram_range=(1,4),analyzer='char')
tfidf_vec_a_2.fit(train['ABSTRACT'].values.tolist() + test['ABSTRACT'].values.tolist())
train_tfidf_a_2 = tfidf_vec_a_2.transform(train['ABSTRACT'].values.tolist())
test_tfidf_a_2 = tfidf_vec_a_2.transform(test['ABSTRACT'].values.tolist())

In [301]:
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    pred_full_test_t_2 = 0
    pred_train_t_2 = np.zeros([train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_tfidf_t_2[dev_index], train_tfidf_t_2[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = mnb(dev_X, dev_y, val_X, val_y, test_tfidf_t_2)
        pred_full_test_t_2 = pred_full_test_t_2 + pred_test_y
        pred_train_t_2[val_index,:] = pred_val_y
#         cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
    pred_full_test_t_2 = pred_full_test_t_2 / 5
    
    pred_full_test_a_2 = 0
    pred_train_a_2 = np.zeros([train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_tfidf_a_2[dev_index], train_tfidf_a_2[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = mnb(dev_X, dev_y, val_X, val_y, test_tfidf_a_2)
        pred_full_test_a_2 = pred_full_test_a_2 + pred_test_y
        pred_train_a_2[val_index,:] = pred_val_y
#         cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
    pred_full_test_a_2 = pred_full_test_a_2 / 5
    train["nb_t_2"+col] = pred_train_t_2[:,0]
    train["nb_t_2"+col] = pred_train_t_2[:,1]
#     train_df["nb_cvec_mws"] = pred_train[:,2]
    test["nb_t_2"+col] = pred_full_test_t_2[:,0]
    test["nb_t_2"+col] = pred_full_test_t_2[:,1]

    train["nb_a_2"+col] = pred_train_a_2[:,0]
    train["nb_a_2"+col] = pred_train_a_2[:,1]
#     train_df["nb_cvec_mws"] = pred_train[:,2]
    test["nb_a_2"+col] = pred_full_test_a_2[:,0]
    test["nb_a_2"+col] = pred_full_test_a_2[:,1]

In [302]:
tfidf_vec_t_2 = TfidfVectorizer(ngram_range=(1,4),analyzer='char')
full_tfidf_t_2 = tfidf_vec_t_2.fit_transform(train['TITLE'].values.tolist() + test['TITLE'].values.tolist())
train_tfidf_t_2 = tfidf_vec_t_2.transform(train['TITLE'].values.tolist())
test_tfidf_t_2 = tfidf_vec_t_2.transform(test['TITLE'].values.tolist())
tfidf_vec_a_2 = TfidfVectorizer(ngram_range=(1,5),analyzer='char')
full_tfidf_a_2 = tfidf_vec_a_2.fit_transform(train['ABSTRACT'].values.tolist() + test['ABSTRACT'].values.tolist())
train_tfidf_a_2 = tfidf_vec_a_2.transform(train['ABSTRACT'].values.tolist())
test_tfidf_a_2 = tfidf_vec_a_2.transform(test['ABSTRACT'].values.tolist())

In [303]:
n_t_2 = 10
n_a_2 =25
svd_t_2 = TruncatedSVD(n_components=n_t_2,algorithm='arpack')
svd_a_2 = TruncatedSVD(n_components=n_a_2,algorithm='arpack')
svd_t_2.fit(full_tfidf_t_2)
svd_a_2.fit(full_tfidf_a_2)
train_svd_t_2 = pd.DataFrame(svd_t_2.transform(train_tfidf_t_2))
test_svd_t_2 = pd.DataFrame(svd_t_2.transform(test_tfidf_t_2))
train_svd_a_2 = pd.DataFrame(svd_a_2.transform(train_tfidf_a_2))
test_svd_a_2 = pd.DataFrame(svd_a_2.transform(test_tfidf_a_2))
train_svd_t_2.columns = ['svd_word_t_2'+str(i) for i in range(n_t_2)]
test_svd_t_2.columns = ['svd_word_t_2'+str(i) for i in range(n_t_2)]
train = pd.concat([train, train_svd_t_2], axis=1)
test = pd.concat([test, test_svd_t_2], axis=1)
train_svd_a_2.columns = ['svd_word_a_2'+str(i) for i in range(n_a_2)]
test_svd_a_2.columns = ['svd_word_a_2'+str(i) for i in range(n_a_2)]
train = pd.concat([train, train_svd_a_2], axis=1)
test = pd.concat([test, test_svd_a_2], axis=1)
# del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [87]:
# for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
#     pred_full_test_a = 0
#     pred_train_a = np.zeros([train.shape[0], 2])
#     kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
#     for dev_index, val_index in kf.split(train):
#         dev_X, val_X = train_tfidf_a[dev_index], train_tfidf_a[val_index]
#         dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
#         pred_val_y, pred_test_y, model = mnb(dev_X, dev_y, val_X, val_y, test_tfidf_a)
#         pred_full_test_a = pred_full_test_a + pred_test_y
#         pred_train_a[val_index,:] = pred_val_y
# #         cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
#     pred_full_test_a = pred_full_test_a / 5
    
    

In [88]:
# train["nb_t_"+col] = pred_train_t[:,0]
# train["nb_t_"+col] = pred_train_t[:,1]
# #     train_df["nb_cvec_mws"] = pred_train[:,2]
# test["nb_t_"+col] = pred_full_test_t[:,0]
# test["nb_t_"+col] = pred_full_test_t[:,1]

# train["nb_a_"+col] = pred_train_a[:,0]
# train["nb_a_"+col] = pred_train_a[:,1]
# #     train_df["nb_cvec_mws"] = pred_train[:,2]
# test["nb_a_"+col] = pred_full_test_a[:,0]
# test["nb_a_"+col] = pred_full_test_a[:,1]
# #     test_df["nb_cvec_mws"] = pred_full_test[:,2]

In [151]:
train.head(20)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,num_words_title,...,nb_t_Physics,nb_a_Physics,nb_t_Mathematics,nb_a_Mathematics,nb_t_Statistics,nb_a_Statistics,nb_t_Quantitative Biology,nb_a_Quantitative Biology,nb_t_Quantitative Finance,nb_a_Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,4,...,2.294661e-01,1.349981e-91,8.358065e-03,2.435926e-107,6.312107e-02,1.000000e+00,1.077997e-04,1.102662e-135,2.203021e-05,3.927251e-252
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,4,...,3.649186e-03,4.582627e-34,8.272478e-06,3.486937e-43,6.875600e-02,7.153414e-01,6.324960e-06,9.184346e-55,6.403780e-09,1.147323e-83
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,8,...,2.352146e-03,8.651439e-15,9.994948e-01,1.000000e+00,3.477615e-01,8.757600e-12,2.610139e-04,1.278738e-40,3.224106e-05,1.875691e-50
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,9,...,1.344681e-02,1.842891e-09,9.997706e-01,1.000000e+00,4.734992e-06,1.124711e-28,6.732965e-09,3.372085e-68,2.630909e-09,1.581201e-81
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,17,...,5.028094e-07,1.064117e-37,2.159575e-06,1.851457e-52,9.992525e-01,1.000000e+00,1.446716e-07,4.239707e-74,1.337440e-12,3.588775e-117
5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0,0,1,0,0,0,9,...,7.430016e-03,4.026017e-50,8.325210e-01,1.000000e+00,5.003247e-02,9.255756e-45,2.797581e-04,8.546183e-120,1.087253e-04,3.252726e-129
6,7,On the rotation period and shape of the hyperb...,We observed the newly discovered hyperbolic ...,0,1,0,0,0,0,15,...,9.999917e-01,1.000000e+00,2.725824e-03,1.059311e-25,2.744245e-04,1.316961e-23,5.015009e-05,1.597326e-38,3.751957e-05,1.787002e-53
7,8,Adverse effects of polymer coating on heat tra...,The ability of metallic nanoparticles to sup...,0,1,0,0,0,0,11,...,9.999999e-01,1.000000e+00,1.506298e-06,1.269417e-50,8.112635e-07,1.677382e-45,2.571034e-07,9.258073e-61,5.843262e-09,7.981515e-95
8,9,SPH calculations of Mars-scale collisions: the...,We model large-scale ($\approx$2000km) impac...,0,1,0,0,0,0,14,...,9.999996e-01,1.000000e+00,2.820553e-05,2.028161e-43,1.592316e-06,2.683779e-40,3.017515e-07,2.018318e-68,3.841754e-09,2.195194e-99
9,10,$\mathcal{R}_{0}$ fails to predict the outbrea...,Time varying susceptibility of host at indiv...,0,0,0,0,1,0,12,...,1.869764e-03,4.911057e-07,5.550554e-02,1.051431e-22,9.390527e-01,2.868221e-23,1.338495e-03,4.050261e-55,1.096564e-05,3.571229e-93


In [141]:
train.columns

Index(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance',
       'num_words_title', 'num_words_abs', 'word_upper_title',
       'word_upper_abs', 'word_title_title', 'word_title_abs', 'svd_word_t0',
       'svd_word_t1', 'svd_word_t2', 'svd_word_t3', 'svd_word_t4',
       'svd_word_t5', 'svd_word_t6', 'svd_word_t7', 'svd_word_a0',
       'svd_word_a1', 'svd_word_a2', 'svd_word_a3', 'svd_word_a4',
       'svd_word_a5', 'svd_word_a6', 'svd_word_a7', 'svd_word_a8',
       'svd_word_a9', 'svd_word_a10', 'svd_word_a11', 'svd_word_a12',
       'svd_word_a13', 'svd_word_a14', 'svd_word_a15', 'svd_word_a16',
       'svd_word_a17', 'svd_word_a18', 'svd_word_a19', 'nb_t_Computer Science',
       'nb_a_Computer Science', 'nb_t_Physics', 'nb_a_Physics',
       'nb_t_Mathematics', 'nb_a_Mathematics', 'nb_t_Statistics',
       'nb_a_Statistics', 'nb_t_Quantitative Biology',
       'nb_a_Quantitative Biology', 

In [374]:
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.7):
    param = {}
    param['objective'] = 'binary:hinge'
    param['eta'] = 0.05
    param['max_depth'] = 14
    param['silent'] = 1
    param['eval_metric'] = "logloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.6
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    param['n_jobs'] = -1
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

In [393]:
train_x = train.drop(columns=['ID','TITLE','ABSTRACT','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])
preds = pd.DataFrame(np.zeros(test.shape[0]))
pred_train_xg = pd.DataFrame(np.zeros(train.shape[0]))
test_x = test.drop(columns=['ID','TITLE','ABSTRACT'])
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train_xgb = np.zeros([train.shape[0]])
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_x.loc[dev_index], train_x.loc[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_x, seed_val=0)
        pred_full_test = pred_full_test + pred_test_y
        pred_train_xgb[val_index] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    preds[col] = pred_full_test//3
    pred_train_xg[col] = pred_train_xgb



[23:44:37] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:21.75071	test-logloss:21.71845
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 50 rounds.
[20]	train-logloss:2.99746	test-logloss:4.76874
[40]	train-logloss:0.90473	test-logloss:4.44379
[60]	train-logloss:0.17348	test-logloss:4.50527
[80]	train-logloss:0.03733	test-logloss:4.50527
Stopping. Best iteration:
[30]	train-logloss:1.73699	test-logloss:4.36475



C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[23:44:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:21.73973	test-logloss:21.76236
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 50 rounds.
[20]	train-logloss:2.85253	test-logloss:5.63817
[40]	train-logloss:0.92010	test-logloss:4.99707
[60]	train-logloss:0.20203	test-logloss:4.77752
[80]	train-logloss:0.04392	test-logloss:5.02342
[100]	train-logloss:0.00659	test-logloss:5.04977
Stopping. Best iteration:
[60]	train-logloss:0.20203	test-logloss:4.77752



C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[23:44:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:21.74722	test-logloss:21.73242
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 50 rounds.
[20]	train-logloss:2.76014	test-logloss:5.35842
[40]	train-logloss:0.81465	test-logloss:4.92799
[60]	train-logloss:0.18445	test-logloss:5.04218
[80]	train-logloss:0.02855	test-logloss:5.14760
Stopping. Best iteration:
[40]	train-logloss:0.81465	test-logloss:4.92799



...

C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[23:49:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:36.39543	test-logloss:36.43751
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 50 rounds.
[20]	train-logloss:0.18445	test-logloss:0.38651
[40]	train-logloss:0.18664	test-logloss:0.38651
[60]	train-logloss:0.09222	test-logloss:0.36894
[80]	train-logloss:0.02196	test-logloss:0.36894
[100]	train-logloss:0.00439	test-logloss:0.36894
[120]	train-logloss:0.00220	test-logloss:0.36894
[140]	train-logloss:0.00000	test-logloss:0.35137
Stopping. Best iteration:
[94]	train-logloss:0.00878	test-logloss:0.34259



C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[23:49:54] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:36.40422	test-logloss:36.40237
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 50 rounds.
[20]	train-logloss:0.17347	test-logloss:0.38651
[40]	train-logloss:0.15371	test-logloss:0.34259
[60]	train-logloss:0.07027	test-logloss:0.35137
[80]	train-logloss:0.00878	test-logloss:0.36016
[100]	train-logloss:0.00000	test-logloss:0.34259
[120]	train-logloss:0.00000	test-logloss:0.34259
Stopping. Best iteration:
[70]	train-logloss:0.03733	test-logloss:0.33380

cv scores :  [nan, nan, nan, nan, nan]


C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\120638\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [314]:
test.head()

,ID,TITLE,ABSTRACT,num_words_title,num_words_abs,num_unique_title,num_unique_abs,word_upper_title,word_upper_abs,word_title_title,...,svd_word_a_215,svd_word_a_216,svd_word_a_217,svd_word_a_218,svd_word_a_219,svd_word_a_220,svd_word_a_221,svd_word_a_222,svd_word_a_223,svd_word_a_224
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...,7,96,7,75,0,0,5,...,0.029011,-0.019336,-0.015417,0.011438,-0.026391,-0.028771,0.017370,-0.062069,-0.044792,-0.012903
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...,14,135,13,96,0,0,2,...,0.014684,0.006344,0.000462,0.034874,-0.008300,0.010092,0.024814,0.007854,-0.000325,0.000300
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...,7,126,7,85,1,7,4,...,0.029525,-0.034381,-0.020763,-0.025617,-0.001453,-0.017915,0.008269,0.014997,-0.033013,0.013496
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...,11,268,11,164,1,13,2,...,0.034751,0.028613,0.034634,-0.008034,-0.022068,0.023631,-0.008512,0.008492,-0.030776,0.023139
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...,11,119,11,78,0,0,7,...,-0.008500,0.016019,-0.023167,0.006702,0.019665,-0.009444,-0.048115,-0.045064,-0.010259,-0.044421


In [392]:
preds.head(30)

,0,Computer Science
0,0.0,0.0
1,0.0,0.0
2,0.0,1.0
3,0.0,0.0
4,0.0,1.0
5,0.0,0.0
6,0.0,0.0
7,0.0,1.0
8,0.0,1.0
9,0.0,0.0


In [394]:
preds.drop(columns=[0],inplace=True)
need = test[['ID']]
preds_final2 = pd.concat([need,preds],axis=1)

In [395]:
preds_final2.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0.0,0.0,0.0,1.0,0.0,0.0
1,20974,0.0,1.0,0.0,0.0,0.0,0.0
2,20975,1.0,0.0,0.0,0.0,0.0,0.0
3,20976,0.0,1.0,0.0,0.0,0.0,0.0
4,20977,1.0,0.0,0.0,0.0,0.0,0.0


In [396]:
print(test.shape)
print(preds.shape)
print(need.shape)

(8989, 109)
(8989, 6)
(8989, 1)


In [397]:
sub = pd.read_csv('sample_submission.csv')
sub.shape
need.head()

,ID
0,20973
1,20974
2,20975
3,20976
4,20977


In [398]:
preds_final2.set_index('ID',inplace=True)
preds_final2.to_csv('srk.csv')

In [321]:
preds_final2.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,
20973,0.0,0.0,0.0,1.0,0.0,0.0
20974,0.0,1.0,0.0,0.0,0.0,0.0
20975,1.0,0.0,0.0,0.0,0.0,0.0
20976,0.0,1.0,0.0,0.0,0.0,0.0
20977,1.0,0.0,0.0,0.0,0.0,0.0


In [399]:
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.7):
    param = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'learning_rate': 0.05,
            'feature_fraction': 0.6,
            'bagging_fraction': 0.7,
            'bagging_freq' : 5,
            'max_depth' : 12,
            'min_child_weight' : 1 ,
            'n_jobs' : -1
            }
 
    
    lgb_train = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgb_eval = lgb.Dataset(test_X, label=test_y)
#         watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = lgb.train(param,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
        
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
    
    print(test_x.shape)
    print(test_X2.shape)

    pred_test_y = model.predict(test_X, ntree_limit = model.best_iteration)
    if test_X2 is not None:
        lgbtest2 = test_X2
        pred_test_y2 = model.predict(lgbtest2, ntree_limit = model.best_iteration)
    return pred_test_y, pred_test_y2, model

In [400]:
train_x = train.drop(columns=['ID','TITLE','ABSTRACT','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])
preds_lg = pd.DataFrame(np.zeros(test.shape[0]))
pred_train_l = pd.DataFrame(np.zeros(train.shape[0]))
test_ = test.drop(columns=['ID','TITLE','ABSTRACT'])
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train_lg = np.zeros([train.shape[0]])
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_x.loc[dev_index], train_x.loc[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = runLGB(dev_X, dev_y, val_X, val_y, test_, seed_val=0)
        pred_full_test = pred_full_test + pred_test_y
        pred_train_lg[val_index] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    preds_lg[col] = pred_full_test//3
    pred_train_l[col] = pred_train_lg
    

Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.376895	valid_1's binary_logloss: 0.383051
[40]	training's binary_logloss: 0.294351	valid_1's binary_logloss: 0.310271
[60]	training's binary_logloss: 0.259807	valid_1's binary_logloss: 0.287444
[80]	training's binary_logloss: 0.237332	valid_1's binary_logloss: 0.277902
[100]	training's binary_logloss: 0.220581	valid_1's binary_logloss: 0.274589
[120]	training's binary_logloss: 0.206899	valid_1's binary_logloss: 0.272151
[140]	training's binary_logloss: 0.19454	valid_1's binary_logloss: 0.272694
Early stopping, best iteration is:
[130]	training's binary_logloss: 0.200381	valid_1's binary_logloss: 0.271937
(8989, 106)
(8989, 106)
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.37392	valid_1's binary_logloss: 0.393192
[40]	training's binary_logloss: 0.289813	valid_1's binary_logloss: 0.325784
[60]	training's binary_logloss: 0.254784	valid_1's binary

[134]	training's binary_logloss: 0.125791	valid_1's binary_logloss: 0.210158
(8989, 106)
(8989, 106)
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.293278	valid_1's binary_logloss: 0.304092
[40]	training's binary_logloss: 0.215673	valid_1's binary_logloss: 0.240865
[60]	training's binary_logloss: 0.182136	valid_1's binary_logloss: 0.221268
[80]	training's binary_logloss: 0.161786	valid_1's binary_logloss: 0.215063
[100]	training's binary_logloss: 0.146526	valid_1's binary_logloss: 0.213297
Early stopping, best iteration is:
[98]	training's binary_logloss: 0.147891	valid_1's binary_logloss: 0.213098
(8989, 106)
(8989, 106)
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.290638	valid_1's binary_logloss: 0.310559
[40]	training's binary_logloss: 0.213033	valid_1's binary_logloss: 0.248612
[60]	training's binary_logloss: 0.179349	valid_1's binary_logloss: 0.231081
[80]	training's binary_logloss: 0

Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.0673573	valid_1's binary_logloss: 0.0917869
[40]	training's binary_logloss: 0.0467235	valid_1's binary_logloss: 0.0837475
[60]	training's binary_logloss: 0.0351966	valid_1's binary_logloss: 0.0809596
[80]	training's binary_logloss: 0.0272177	valid_1's binary_logloss: 0.0806443
Early stopping, best iteration is:
[75]	training's binary_logloss: 0.0290078	valid_1's binary_logloss: 0.0801908
(8989, 106)
(8989, 106)
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.0684449	valid_1's binary_logloss: 0.090831
[40]	training's binary_logloss: 0.0481947	valid_1's binary_logloss: 0.0831117
[60]	training's binary_logloss: 0.0363344	valid_1's binary_logloss: 0.0804505
[80]	training's binary_logloss: 0.0282094	valid_1's binary_logloss: 0.0800484
Early stopping, best iteration is:
[72]	training's binary_logloss: 0.0311617	valid_1's binary_logloss: 0.0798002
(8989

In [404]:
# preds_lg.drop(columns=[0],inplace=True)
need = test[['ID']]
preds_final2_lg = pd.concat([need,preds_lg],axis=1)

In [405]:
preds_final2_lg.set_index('ID',inplace=True)
preds_final2_lg.to_csv('srk_lg.csv')

In [409]:
def runCB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.7):
    param = {
            'boosting_type': 'gbdt',
            'objective': 'binary:hinge',
            'metric': {'binary_logloss'},
            'learning_rate': 0.05,
            'feature_fraction': 0.7,
            'bagging_fraction': 0.6,
            'bagging_freq' : 5,
            'max_depth' : 10,
            'min_child_weight' : 1 
            }
 
    
    lgb_train = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        model = CatBoostClassifier(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='Logloss',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
        model.fit(train_X,train_y,eval_set=(test_X,test_y),use_best_model=True,verbose=50)
    
        
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(test_X)
    if test_X2 is not None:
        lgbtest2 = test_X2
        pred_test_y2 = model.predict(lgbtest2)
    return pred_test_y, pred_test_y2, model

In [410]:
train_x = train.drop(columns=['ID','TITLE','ABSTRACT','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])
preds_cb = pd.DataFrame(np.zeros(test.shape[0]))
preds_train_c = pd.DataFrame(np.zeros(train.shape[0]))
test__ = test.drop(columns=['ID','TITLE','ABSTRACT'])
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([train.shape[0]])
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_x.loc[dev_index], train_x.loc[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = runCB(dev_X, dev_y, val_X, val_y, test__, seed_val=0)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    preds_cb[col] = pred_full_test//3
    preds_train_c = pred_train

0:	learn: 0.6469417	test: 0.6475127	best: 0.6475127 (0)	total: 436ms	remaining: 3m 37s
50:	learn: 0.2425304	test: 0.2921967	best: 0.2921967 (50)	total: 22.8s	remaining: 3m 20s
100:	learn: 0.1897402	test: 0.2818803	best: 0.2818803 (100)	total: 44.6s	remaining: 2m 56s
150:	learn: 0.1491950	test: 0.2795479	best: 0.2793050 (147)	total: 1m 6s	remaining: 2m 32s
200:	learn: 0.1201979	test: 0.2794450	best: 0.2786520 (183)	total: 1m 28s	remaining: 2m 11s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2786519677
bestIteration = 183

Shrink model to first 184 iterations.


0:	learn: 0.6463561	test: 0.6493867	best: 0.6493867 (0)	total: 437ms	remaining: 3m 38s
50:	learn: 0.2365874	test: 0.3123554	best: 0.3123554 (50)	total: 22.7s	remaining: 3m 19s
100:	learn: 0.1836050	test: 0.3039380	best: 0.3039380 (100)	total: 44.7s	remaining: 2m 56s
150:	learn: 0.1475733	test: 0.3023238	best: 0.3021931 (143)	total: 1m 7s	remaining: 2m 34s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3018145363
bestIteration = 167

Shrink model to first 168 iterations.


0:	learn: 0.6485288	test: 0.6506632	best: 0.6506632 (0)	total: 433ms	remaining: 3m 35s
50:	learn: 0.2370344	test: 0.3090650	best: 0.3090650 (50)	total: 22.6s	remaining: 3m 18s
100:	learn: 0.1845596	test: 0.3044139	best: 0.3044139 (100)	total: 44.1s	remaining: 2m 54s
150:	learn: 0.1461442	test: 0.3034377	best: 0.3033201 (141)	total: 1m 5s	remaining: 2m 32s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.303150798
bestIteration = 154

Shrink model to first 155 iterations.


0:	learn: 0.6475451	test: 0.6481587	best: 0.6481587 (0)	total: 462ms	remaining: 3m 50s
50:	learn: 0.2446659	test: 0.2870280	best: 0.2870280 (50)	total: 23.2s	remaining: 3m 24s
100:	learn: 0.1916911	test: 0.2746485	best: 0.2746485 (100)	total: 44.9s	remaining: 2m 57s
150:	learn: 0.1533378	test: 0.2704675	best: 0.2704675 (150)	total: 1m 7s	remaining: 2m 36s
200:	learn: 0.1276952	test: 0.2693811	best: 0.2693811 (200)	total: 1m 31s	remaining: 2m 15s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2687546716
bestIteration = 219

Shrink model to first 220 iterations.


0:	learn: 0.6467003	test: 0.6484844	best: 0.6484844 (0)	total: 588ms	remaining: 4m 53s
50:	learn: 0.2385191	test: 0.3057885	best: 0.3057885 (50)	total: 22.7s	remaining: 3m 20s
100:	learn: 0.1860881	test: 0.2985312	best: 0.2985148 (98)	total: 45.3s	remaining: 2m 59s
150:	learn: 0.1500219	test: 0.2967548	best: 0.2966876 (149)	total: 1m 8s	remaining: 2m 37s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2966514458
bestIteration = 151

Shrink model to first 152 iterations.
cv scores :  [4.124953431856543, 4.495466544797175, 4.51300956294856, 3.9529952656069036, 4.430644913690149]


0:	learn: 0.6259459	test: 0.6276147	best: 0.6276147 (0)	total: 457ms	remaining: 3m 48s
50:	learn: 0.1279789	test: 0.1695624	best: 0.1695624 (50)	total: 22.5s	remaining: 3m 17s
100:	learn: 0.0899026	test: 0.1619207	best: 0.1618967 (97)	total: 44s	remaining: 2m 53s
150:	learn: 0.0663918	test: 0.1606665	best: 0.1606665 (150)	total: 1m 5s	remaining: 2m 31s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.1606466135
bestIteration = 152

Shrink model to first 153 iterations.


0:	learn: 0.6266727	test: 0.6265981	best: 0.6265981 (0)	total: 424ms	remaining: 3m 31s
50:	learn: 0.1292994	test: 0.1610201	best: 0.1610201 (50)	total: 21.9s	remaining: 3m 12s
100:	learn: 0.0926176	test: 0.1542161	best: 0.1542161 (100)	total: 44.1s	remaining: 2m 54s
150:	learn: 0.0697602	test: 0.1521791	best: 0.1521467 (149)	total: 1m 6s	remaining: 2m 33s
200:	learn: 0.0534481	test: 0.1509669	best: 0.1509532 (198)	total: 1m 29s	remaining: 2m 12s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.1507261618
bestIteration = 224

Shrink model to first 225 iterations.


0:	learn: 0.6250858	test: 0.6278066	best: 0.6278066 (0)	total: 472ms	remaining: 3m 55s
50:	learn: 0.1264327	test: 0.1755952	best: 0.1755952 (50)	total: 22.4s	remaining: 3m 17s
100:	learn: 0.0881040	test: 0.1692414	best: 0.1691872 (99)	total: 45.8s	remaining: 3m 1s


KeyboardInterrupt: 

In [ ]:
def runRF(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.7):
    param = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}
            
 
    
#     lgb_train = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        model = RandomForestClassifier('n_jobs': -1,
    'n_estimators': 50,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0)
        model.fit(train_X,train_y)
    
        
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(test_X)
    if test_X2 is not None:
        lgbtest2 = test_X2
        pred_test_y2 = model.predict(lgbtest2)
    return pred_test_y, pred_test_y2, model

In [ ]:
train_x = train.drop(columns=['ID','TITLE','ABSTRACT','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])
preds_rf = pd.DataFrame(np.zeros(test.shape[0]))
test___ = test.drop(columns=['ID','TITLE','ABSTRACT'])
for col in ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']:
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train_rf = np.zeros([train.shape[0]])
    for dev_index, val_index in kf.split(train):
        dev_X, val_X = train_x.loc[dev_index], train_x.loc[val_index]
        dev_y, val_y = train.loc[dev_index,col], train.loc[val_index,col]
        pred_val_y, pred_test_y, model = runCB(dev_X, dev_y, val_X, val_y, test___, seed_val=0)
        pred_full_test = pred_full_test + pred_test_y
        pred_train_rf[val_index] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    preds_rf[col] = pred_full_test//3

In [ ]:
train_cs = pd.concat([pred_train_cb[:,'Computer Science'],pred_train_xgb[:,1],pred_train_lg[:,1],pred_train_rf[:,1]],axis=1)
train_phy = pd.concat([pred_train_cb[:,'Computer Science'],pred_train_xgb[:,2],pred_train_lg[:,2],pred_train_rf[:,2]],axis=1)
train_mat = pd.concat([pred_train_cb[:,'Computer Science'],pred_train_xgb[:,3],pred_train_lg[:,3],pred_train_rf[:,3]],axis=1)
train_sta = pd.concat([pred_train_cb[:,'Computer Science'],pred_train_xgb[:,4],pred_train_lg[:,4],pred_train_rf[:,4]],axis=1)
train_qb = pd.concat([pred_train_cb[:,'Computer Science'],pred_train_xgb[:,5],pred_train_lg[:,5],pred_train_rf[:,5]],axis=1)
train_qf = pd.concat([pred_train_cb[:,6],pred_train_xgb[:,6],pred_train_lg[:,6],pred_train_rf[:,6]],axis=1)

test_cs = pd.concat([preds_cb[:,'Computer Science'],preds[:,'Computer Science'],preds_lg[:,'Computer Science'],preds_rf[:,'Computer Science']],axis=1)
test_phy = pd.concat([preds_cb[:,'Physics'],preds[:,'Physics'],preds_lg[:,'Physics'],preds_rf[:,'Physics']],axis=1)
test_mat = pd.concat([preds_cb[:,'Mathematics'],preds[:,'Mathematics'],preds_lg[:,'Mathematics'],preds_rf[:,'Mathematics']],axis=1)
test_sta = pd.concat([preds_cb[:,'Statistics'],preds[:,'Statistics'],preds_lg[:,'Statistics'],preds_rf[:,'Statistics']],axis=1)
test_qb = pd.concat([preds_cb[:,'Quantitative Biology'],preds[:,'Quantitative Biology'],preds_lg[:,'Quantitative Biology'],preds_rf[:,'Quantitative Biology']],axis=1)
test_qf = pd.concat([preds_cb[:,'Quantitative Finance'],preds[:,'Quantitative Finance'],preds_lg[:,'Quantitative Finance'],preds_rf[:,'Quantitative Finance']],axis=1)